In [1]:
import pandas as pd
import numpy as np
from icecream import ic

In [2]:
PATH = "../../../Documents/ClassDocs/cee240Docs/p6/CEE240F22_Nov18.xlsx"


In [3]:
# import most recent excel files
# get all before march
# get all during march 
# find percentages 

# import data

In [4]:
df = pd.read_excel(PATH, sheet_name=0, engine="openpyxl", skiprows=[1] )
# rename columns to be easier to access
df.rename(columns={"target_drtn_hr_cnt": "duration", "total_float_hr_cnt": "float"}, inplace=True)

df.drop([
 'status_code',
 'wbs_id', 
 'delete_record_flag'], 
 axis=1, inplace=True)
df = df[['start_date', 'end_date', 'task_name', 'duration', 'float', 'task_code']]

df.head(3)

/Users/julietnwagwuume-ezeoke/ClassCode/CEE240local/cee240env/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,start_date,end_date,task_name,duration,float,task_code
0,2022-09-01 08:00:00,NaT,NTP,0,0,P1B1000
1,2022-09-01 08:00:00,2022-09-01 16:00:00,submit shut down request for facilities,1,0,P1B1010
2,2022-09-02 08:00:00,2022-09-05 16:00:00,"demo ceilings GL 6.5, M.4",2,310,P1B1020


## make appropriate dfs

In [5]:
mar23_start = pd.to_datetime("2023/03/01")
mar23_end = pd.to_datetime("2023/04/01")

In [6]:
df_mar_start = df[df["end_date"] < mar23_start].reset_index(drop=True)
df_mar_start

,start_date,end_date,task_name,duration,float,task_code
0,2022-09-01 08:00:00,2022-09-01 16:00:00,submit shut down request for facilities,1,0,P1B1010
1,2022-09-02 08:00:00,2022-09-05 16:00:00,"demo ceilings GL 6.5, M.4",2,310,P1B1020
2,2022-09-15 08:00:00,2022-09-19 16:00:00,"erect temp barrier, GL 6.5, M.4",3,0,P1B1030
3,2022-09-20 08:00:00,2022-09-22 16:00:00,demo exterior walls GL 5.7 M,3,0,P1B1040
4,2022-09-23 08:00:00,2022-09-27 16:00:00,demo interior walls GL 5.7 - 7.1 M.4,3,0,P1B1050
...,...,...,...,...,...,...
90,2023-01-13 08:00:00,2023-01-18 16:00:00,"install cieling finsihes 1L, Sector A",4,210,P2A1L1130
91,2023-01-19 08:00:00,2023-01-23 16:00:00,"testing and commissioning 1L, Sector A",3,210,P2A1L1140
92,NaT,2023-02-01 08:00:00,Staff Relocation (A_L1),0,0,M1010
93,2022-11-22 08:00:00,2022-11-28 16:00:00,Begin MEPF Start Up,5,250,M2030


In [7]:
df_mar_end = df[(df["end_date"] >= mar23_start) &  (df["end_date"] < mar23_end)].reset_index(drop=True)
df_mar_end

,start_date,end_date,task_name,duration,float,task_code
0,2023-02-27 08:00:00,2023-03-02 16:00:00,"erect columns 2L, Sector B",4,0,P1B1440
1,2023-03-03 08:00:00,2023-03-08 16:00:00,"erect steel deck 3L, Sector B",4,0,P1B1450
2,2023-03-09 08:00:00,2023-03-28 16:00:00,"curtain wall GL, Sector B",14,3,P1B1460
3,2023-03-09 08:00:00,2023-03-28 16:00:00,"curtain wall 1L, Sector B",14,0,P1B1470
4,2023-03-09 08:00:00,2023-03-28 16:00:00,"curtain wall 2L, Sector B",14,3,P1B1480
5,2023-03-29 08:00:00,2023-03-31 16:00:00,"erect temp barrier, west of new main connector...",3,3,P1B1490
6,2023-03-29 08:00:00,2023-03-31 16:00:00,"erect temp barrier, west of new main connector...",3,0,P1B1500


# find percentages of work done to date

In [8]:
# list(df["task_name"])

## import work descriptions

In [10]:
from cost_categories import cd

In [12]:
cid = {} # category index dict
for k,v in cd.items():
    # iterate over list in v, find all rows in dataframe that match...
    arr = []
    if type(v) != int:
        for i in v:
            v_ix = list(df.loc[df["task_name"].str.contains(i)].index)
            arr.extend(v_ix)
        cid[k] = list(arr)
    else:
        cid[k] = []

In [28]:
comd = {} # completion dict 
for k,v in cid.items():
    if len(v) > 0: 
        total_duration = df.iloc[cid[k]]["duration"].sum()

        completed_duration = pd.merge(df.iloc[cid[k]], df_mar_start, on=['task_code'], how='inner')["duration_x"].sum()

        march_duration = pd.merge(df.iloc[cid[k]], df_mar_end, on=['task_code'], how='inner')["duration_x"].sum()

        completed_percent = np.round((completed_duration/total_duration), 3)
        march_percent = np.round(((march_duration + completed_duration)/total_duration), 3)
        comd[k] = (completed_percent, march_percent)
    else:
        comd[k] = (0,0)

## general, indirect work percentages

In [43]:
# number of months in project 
df["end_date"].sort_values().iloc[-2]

Timestamp('2023-11-13 16:00:00')

In [53]:
df["end_date"].iloc[-1]
start_month = df["end_date"].sort_values().iloc[0]
end_month = df["end_date"].sort_values().iloc[-2]


In [64]:
def month_diff(a, b):
    return 12 * (a.year - b.year) + (a.month - b.month)

In [65]:
project_len = month_diff(end_month, start_month)
project_len


14

In [69]:
len_march23_start = month_diff(mar23_start, start_month)
print(len_march23_start, len_march23_start/project_len)

6 0.42857142857142855


In [70]:
len_march23_end = month_diff(mar23_end, start_month)
print(len_march23_end, len_march23_end/project_len)

7 0.5


# export 

In [29]:
percdf = pd.DataFrame(comd).T
percdf.head()

,0,1
BP 1A Structural Concrete,1.000,1.000
BP 1B Exterior Glass & GFRC,0.000,1.000
BP 1C Shoring,1.000,1.000
BP 1D HVAC & Plumbing,0.566,0.566
BP 1E Electrical,0.569,0.569


In [30]:
percdf.to_csv("outputs/percentage.csv")